In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import string
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from torch.utils.data import DataLoader
from tqdm import tqdm
!pip install adabelief-pytorch==0.2.0
from adabelief_pytorch import AdaBelief
from torch.optim import *
import math
from sklearn.metrics import f1_score

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
# import os
# os.listdir('../input/emnist')
# os.listdir('data/EMNIST/processed/')

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5))])

train_set = torchvision.datasets.EMNIST(root='./data', train=True, download=True, transform=transform, split='bymerge')
test_set = torchvision.datasets.EMNIST(root='./data', train=False, download=False, transform=transform, split='bymerge')

cuda:0


Extracting ./data/EMNIST/raw/emnist.zip to ./data/EMNIST/raw
Processing byclass


/opt/conda/lib/python3.7/site-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729138878/work/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Processing bymerge
Processing balanced
Processing letters
Processing digits
Processing mnist
Done!


In [4]:
class ResNet(nn.Module):
    
    def __init__(self, res, linear_dim=128, output_dim=47):
        
        super().__init__()
        self.resnet = res
        self.linear = nn.Linear(linear_dim, output_dim)
    
    def forward(self, x):
        
        x = self.resnet(x)
        x = self.linear(x.squeeze())
        
        return x.squeeze(1)

In [5]:
def evaluate(model, valid_dl):

  # we don't want to add evaluation related computation to computation graph
  with torch.no_grad():
    criterion = nn.CrossEntropyLoss()
    valid_loss_lst = torch.tensor([]).to('cuda')
    y_true_lst = torch.tensor([]).to('cuda')
    y_pred_lst = torch.tensor([]).to('cuda')
    
    for X_valid, y_valid in valid_dl:
      X_valid = X_valid.to('cuda')
      y_valid = y_valid.to('cuda')
      pred = model(X_valid)
      valid_loss_lst = torch.cat((valid_loss_lst, criterion(pred, y_valid).view(-1)))
      predicted = pred.argmax(1).squeeze()
      y_pred_lst = torch.cat((y_pred_lst, predicted))
      y_true_lst = torch.cat((y_true_lst, y_valid))

  return valid_loss_lst.mean().item(), f1_score(y_pred=y_pred_lst.to('cpu'), y_true=y_true_lst.to('cpu'), average='micro')


def train_res(**params):

  # define model and move it to gpu
  model = params['model']
  model.to('cuda')

  # define optimizer
  opt = params['optimizer'][0](model.parameters(), **params['optimizer'][1])

  # define data loader
  
  train_dl = DataLoader(params['train_data'], batch_size=params['batch_size'])
  valid_dl = DataLoader(params['test_data'], batch_size=params['batch_size'])

  # define loss
  criterion = torch.nn.CrossEntropyLoss()

  # start training
  valid_loss_lst = []
  train_loss_lst = []
  train_f1_lst = []
  valid_f1_lst = []

  for e in tqdm(range(params['epochs'] + 1)):
    train_losses = torch.tensor([]).to('cuda')
    correct = 0
    y_true_lst = torch.tensor([]).to('cuda')
    y_pred_lst = torch.tensor([]).to('cuda')
    
    for X_train, y_train in train_dl:

      X_train = X_train.to('cuda')
      y_train = y_train.to('cuda')
        
      pred = model(X_train)
      loss = criterion(pred, y_train)

      # perform backward propagation
      loss.backward()

      # update model parameters and clear up gradient after update
      opt.step()
      opt.zero_grad()
      train_losses = torch.cat((train_losses, loss.view(-1)))

      with torch.no_grad():
        predicted = pred.argmax(1).squeeze()
        y_pred_lst = torch.cat((y_pred_lst, predicted))
        y_true_lst = torch.cat((y_true_lst, y_train))
      
        
    valid_loss, valid_f1 = evaluate(model, valid_dl)
    train_loss = train_losses.mean().item()
    train_f1 = f1_score(y_pred=y_pred_lst.to('cpu'), y_true=y_true_lst.to('cpu'), average='micro')
    train_loss_lst.append(train_loss)
    valid_loss_lst.append(valid_loss)
    train_f1_lst.append(train_f1)
    valid_f1_lst.append(valid_f1)
    

    print(f'epoch: {e}, training loss: {train_loss}, training f1: {train_f1}, validation loss: {valid_loss}, validation f1: {valid_f1}')

  return model, train_loss_lst, valid_loss_lst, train_f1_lst, valid_f1_lst



In [6]:
class Padam(Optimizer):
    """Implements Partially adaptive momentum estimation (Padam) algorithm.
        params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
        lr (float, optional): learning rate (default: 1e-1)
        betas (Tuple[float, float], optional): coefficients used for computing
            running averages of gradient and its square (default: (0.9, 0.999))
        eps (float, optional): term added to the denominator to improve
            numerical stability (default: 1e-8)
        weight_decay (float, optional): weight decay (L2 penalty) (default: 0)
        partial (float, optional): partially adaptive parameter
    """

    def __init__(self, params, lr=1e-1, betas=(0.9, 0.999), eps=1e-8, weight_decay=0, amsgrad=True, partial = 1/4):
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter at index 0: {}".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter at index 1: {}".format(betas[1]))
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay, amsgrad=amsgrad, partial = partial)
        super(Padam, self).__init__(params, defaults)

    def step(self, closure=None):
        """Performs a single optimization step.
        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                if grad.is_sparse:
                    raise RuntimeError('Adam does not support sparse gradients, please consider SparseAdam instead')
                amsgrad = group['amsgrad']
                partial = group['partial']

                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state['step'] = 0
                    # Exponential moving average of gradient values
                    state['exp_avg'] = torch.zeros_like(p.data)
                    # Exponential moving average of squared gradient values
                    state['exp_avg_sq'] = torch.zeros_like(p.data)
                    if amsgrad:
                        # Maintains max of all exp. moving avg. of sq. grad. values
                        state['max_exp_avg_sq'] = torch.zeros_like(p.data)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                if amsgrad:
                    max_exp_avg_sq = state['max_exp_avg_sq']
                beta1, beta2 = group['betas']

                state['step'] += 1

                if group['weight_decay'] != 0:
                    grad = grad.add(group['weight_decay'], p.data)

                # Decay the first and second moment running average coefficient
                exp_avg.mul_(beta1).add_(1 - beta1, grad)
                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                if amsgrad:
                    # Maintains the maximum of all 2nd moment running avg. till now
                    torch.max(max_exp_avg_sq, exp_avg_sq, out=max_exp_avg_sq)
                    # Use the max. for normalizing running avg. of gradient
                    denom = max_exp_avg_sq.sqrt().add_(group['eps'])
                else:
                    denom = exp_avg_sq.sqrt().add_(group['eps'])

                bias_correction1 = 1 - beta1 ** state['step']
                bias_correction2 = 1 - beta2 ** state['step']
                step_size = group['lr'] * math.sqrt(bias_correction2) / bias_correction1

                p.data.addcdiv_(-step_size, exp_avg, denom**(partial*2))
                
        return loss

def get_resnet(state_dict=None):

  resnet18 = models.resnet18()
  emnist_cov2d = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  new_resnet18 = nn.Sequential(*[emnist_cov2d, resnet18.bn1, resnet18.relu, resnet18.maxpool] + list(resnet18.layer1) + [resnet18.avgpool])
  out_resnet = ResNet(new_resnet18, 64)

  if state_dict is not None:
    out_resnet.load_state_dict(state_dict)
  
  return out_resnet

In [7]:
my_resnet_adam = get_resnet()
default_params = my_resnet_adam.state_dict()
my_resnet_adam_belief = get_resnet(default_params)
my_resnet_padam = get_resnet(default_params)
my_resnet_SGD = get_resnet(default_params)

# Momentum

In [8]:
params = {
    'optimizer': (torch.optim.SGD, {'lr': 0.0005, 'momentum': 0.9}),
    'model': my_resnet_SGD,
    'epochs': 30,
    'batch_size': 64,
    'train_data': train_set,
    'test_data': test_set
    
}
model_SGD, train_loss_lst_SGD, valid_loss_lst_SGD, train_f1_lst_SGD, valid_f1_lst_SGD = train_res(**params)

  3%|▎         | 1/31 [02:50<1:25:11, 170.39s/it]

epoch: 0, training loss: 1.2380510568618774, training f1: 0.6810018741080793, validation loss: 0.6047092080116272, validation f1: 0.8188492387575974


  6%|▋         | 2/31 [05:38<1:22:04, 169.81s/it]

epoch: 1, training loss: 0.49346446990966797, training f1: 0.8460838591725268, validation loss: 0.43002229928970337, validation f1: 0.8608873567566173


 10%|▉         | 3/31 [08:26<1:19:00, 169.29s/it]

...

epoch: 28, training loss: 0.24547895789146423, training f1: 0.9081572416797051, validation loss: 0.2659539580345154, validation f1: 0.9007934802231717


 97%|█████████▋| 30/31 [1:24:26<02:49, 169.19s/it]

epoch: 29, training loss: 0.24426354467868805, training f1: 0.908499681917436, validation loss: 0.26542407274246216, validation f1: 0.9009052380010832


100%|██████████| 31/31 [1:27:14<00:00, 168.85s/it]

epoch: 30, training loss: 0.2431037425994873, training f1: 0.9088191972856955, validation loss: 0.26492905616760254, validation f1: 0.9010599795397299


In [13]:
import pandas as pd
sgd_df = pd.DataFrame(data={'train_loss': train_loss_lst_SGD, 
                            'valid_loss': valid_loss_lst_SGD, 
                            'train_f1': train_f1_lst_SGD, 
                            'valid_f1': valid_f1_lst_SGD, 
                            'category': ['sgd']*len(train_loss_lst_SGD)})

sgd_df.to_parquet('./sgd_results.parquet')

## ADAM

In [ ]:
params = {
    'optimizer': (torch.optim.Adam, {'lr': 0.0005}),
    'model': my_resnet_adam,
    'epochs': 30,
    'batch_size': 64,
    'train_data': train_set,
    'test_data': test_set
    
}
model, train_loss_lst, valid_loss_lst, train_f1_lst, valid_f1_lst = train_res(**params)

  3%|▎         | 1/31 [02:58<1:29:04, 178.16s/it]

epoch: 0, training loss: 0.4217078983783722, training f1: 0.8625897651920246, validation loss: 0.30546167492866516, validation f1: 0.8898068309792561


  6%|▋         | 2/31 [05:56<1:26:10, 178.29s/it]

epoch: 1, training loss: 0.29105445742607117, training f1: 0.89365296332594, validation loss: 0.2811935544013977, validation f1: 0.8967616034662105


 10%|▉         | 3/31 [08:53<1:23:01, 177.92s/it]

...

epoch: 19, training loss: 0.20134477317333221, training f1: 0.9207429950195721, validation loss: 0.26664745807647705, validation f1: 0.9024956371482854


 68%|██████▊   | 21/31 [1:02:13<29:48, 178.84s/it]

epoch: 20, training loss: 0.19888976216316223, training f1: 0.9215525294727853, validation loss: 0.2685604691505432, validation f1: 0.9022377345838744


 71%|███████   | 22/31 [1:05:12<26:51, 179.07s/it]

epoch: 21, training loss: 0.19666428864002228, training f1: 0.9222101866657497, validation loss: 0.27028027176856995, validation f1: 0.9019798320194631


In [ ]:
adam_df = belif_df = pd.DataFrame(data={'train_loss': train_loss_lst, 
                            'valid_loss': valid_loss_lst, 
                            'train_f1': train_f1_lst, 
                            'valid_f1': valid_f1_lst, 
                            'category': ['adam']*len(train_loss_lst_SGD)})

adam_df.to_parquet('./adam_results.parquet')

# AdaBelif

In [ ]:
params = {
    'optimizer': (AdaBelief, {'lr': 0.0005}),
    'model': my_resnet_adam_belief,
    'epochs': 30,
    'batch_size': 64,
    'train_data': train_set,
    'test_data': test_set
    
}

model_belief, train_loss_lst_belief, valid_loss_lst_belief, train_f1_lst_belief, valid_f1_lst_belief = train_res(**params)

In [ ]:
belif_df = pd.DataFrame(data={'train_loss': train_loss_lst_belief, 
                            'valid_loss': valid_loss_lst_belief, 
                            'train_f1': train_f1_lst_belief, 
                            'valid_f1': valid_f1_lst_belief, 
                            'category': ['belief']*len(train_loss_lst_SGD)})

belif_df.to_parquet('./belif_results.parquet')

# Padam

In [ ]:
params = {
    'optimizer': (Padam, {'lr': 0.0005}),
    'model': my_resnet_padam,
    'epochs': 30,
    'batch_size': 64,
    'train_data': train_set,
    'test_data': test_set
    
}

model_padam, train_loss_lst_padam, valid_loss_lst_padam, train_f1_lst_padam, valid_f1_lst_padam = train_res(**params)

In [ ]:
padam_df = pd.DataFrame(data={'train_loss': train_loss_lst_padam, 
                            'valid_loss': valid_loss_lst_padam, 
                            'train_f1': train_f1_lst_padam, 
                            'valid_f1': valid_f1_lst_padam, 
                            'category': ['padam']*len(train_loss_lst_SGD)})

padam_df.to_parquet('./padam_results.parquet')

In [ ]:
full_df = pd.concat([belif_df, adam_df, padam_df, sgd_df])